In [1]:
# Helper functions 

def createCType():
    """Create a sorted list of unique Terrain."""
    
    c = set()
    for cell in terrain.find():
        c.add(cell["terrain"])

    tmp = list(c)
    
    return sorted(tmp)

def cTypeToIdx(val):
    """Return the index of a given terrain value. Throws a ValueError if the value is not found in the list."""
    
    return CType.index(val)

def getDistribution(idx):
    """
    Return the distribution of a given index. 
    
    Args:
        idx (int): The cell index of a page. The range is 0-175. 0 is the upper left corner.
    """
    
    # Get all of the cells for this index
    results = list()
    for cell in terrain.find({"idx":idx}):
        results.append(cell["terrain"])

    # Calculate the Distribution of values
    uniqueItems = set(results)
    totalLen = float(len(results)) 
    d = {}
    for item in uniqueItems:
        percent = 100*results.count(item) / totalLen
        d[item] = percent
        
    # Sort by Terrain type
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)

def createObservationDisribution(idx1, idx2):
    
    # Create an empty matrix of zeros 
    od = np.zeros([len(CType),len(CType)])
    
    for cell in terrain.find({"idx":idx1}):
        
        # Find the origin cell for this page
        delX, delY = idxToCoord(idx1)
        x = cell["x"]
        y = cell["y"] 
        
        origincell = [x - delX, y - delY]
        print origincell

        # Find the second index for this cell on this page 
#         x = cell["x"]
#         y = cell["y"] 
        
        del2X, del2Y = idxToCoord(idx2)
        print del2X, del2Y
        
        print "Find ", origincell[0] + del2X, origincell[1] + del2Y
        
        cell2 = terrain.find_one({"x":origincell[0] + del2X, "y":origincell[1] + del2Y})
        print cell2
        
        # Fetch the terrain
        Xi = cTypeToIdx(cell["terrain"])            
        Yi = cTypeToIdx(cell2["terrain"])
            
        od[Xi, Yi] += 1
    
    return od

def idxToCoord(idx): 
    x = idx % 16
    y = idx / 16
    return x, y  


def getPairs(idx, idx2):
    
# Validate pairs of cells.
# Show all terrain for cell index 0 and 1
    
    x, y = idxToCoord(idx)

    data = []
    for cell in terrain.find({"idx":idx}):
        
        # find x.y offset between idx 1 and idx2
        x2, y2 = idxToCoord(idx2)
        delx = x2-x
        dely = y2-y
        

        # Find the origin cell for this page  
        cell2 = terrain.find_one({"x":cell["x"] + delx, "y":cell["y"] + dely})

        data.append((cell["terrain"], cell2["terrain"]))
    #     print cell["terrain"], cell2["terrain"]

    df = pd.DataFrame(data)

    # Add Column titles
    df.columns = ["Cell0", "Cell1"]
    df['Count'] = 1
    
    return df

def getMarginal(df, idx):
    mdf = df.groupby(idx).sum()  / df["Count"].sum()
    mdf.columns = ["Prob"] 
    return mdf

def cartisianProduct(marginal0, marginal1):
    l = []
    for t in list(marginal0.index):
        r = []
        for u in list(marginal1.index):
            r.append(marginal0.loc[t]["Prob"] * marginal1.loc[u]["Prob"])
        l.append(r)
        
    df2 = pd.DataFrame(l)
    df2.columns = marginal1.index
    df2.index = marginal0.index

    return df2